In [1]:
#!pip install yfinance
#!pip install vectorbt

In [2]:
import vectorbt as vbt
import pandas_ta as ta
import pandas as pd
import warnings
import numpy as np
from pylab import *
import math
import time
from binance.spot import Spot 
import pandas as pd
import datetime

warnings.simplefilter(action='ignore',category=FutureWarning)
pd.set_option("display.max_rows",1000)

In [40]:
client = Spot()

now_state=[
0,#0倉位有沒有做多了 
0,#1倉位有沒有做空了
0,#2綠ATR在上黃上，(等待收盤價破上黃線or平倉)
0,#3紅ATR在下黃下，(等待收盤價破下黃線or平倉)
0,#4還有沒有倉位
]

#參數
m15=15*60*1000 #15分鐘轉換成毫秒
atr_times=1.5
length=500
data_length=500
h=8
mult=2.5
k=2

while 1:
    t1=time.time()

    #抓資料
    start = time.time()*1000-m15*502
    end = time.time()*1000
    new_start=int(start)
    new_end=int(end)
    df=pd.DataFrame(client.klines("BTCUSDT", "15m",limit=1000,startTime=str(new_start),endTime=str(new_end)))
    df.columns = ['datetime', 'open', 'high', 'low', 'close', 'volume','close_time', 'qav', 'num_trades','taker_base_vol', 'taker_quote_vol', 'ignore']
    df=df[['datetime', 'open', 'high', 'low', 'close']].astype(float)
    df.index = [datetime.datetime.fromtimestamp(x/1000.0) for x in df.datetime]
    df=df.astype(float)

    #算ATR
    atr=ta.atr(df['high'],df['low'],df['close'],length=14)
    df['high_atr']=df['high']+atr*atr_times
    df['low_atr']=df['low']-atr*atr_times

    #計算500根nadaraya
    data=df[['open','high','low','close','high_atr','low_atr']][-500:]
    y = []
    sum_e = 0.
    for i in range(length):
        sum = 0.
        sumw = 0.
        for j in range(length):
            w = math.exp(-(math.pow(i-j,2)/(h*h*2)))
            sum += data['close'].iloc[j]*w
            sumw += w
        y2 = sum/sumw
        sum_e += abs(data['close'].iloc[i] - y2)
        y.append(y2)
    mae = sum_e/length*mult

    second_last_up_y2_mae=y[-2]+mae
    second_last_dn_y2_mae=y[-2]-mae
    second_last_high_atr=df['high_atr'][-2]
    second_last_low_atr=df['low_atr'][-2]
    second_last_close=df['close'][-2]

    print("花時:",time.time()-t1,
        "\n前一根黃色上軌:",second_last_up_y2_mae,
        "\n前一根黃色下軌:",second_last_dn_y2_mae,
        "\n前一根綠色上ATR:",second_last_high_atr,
        "\n前一根紅色下ATR:",second_last_low_atr,
        "\n前一根收盤:",second_last_close,
        "\n當前狀態:","有做多=",now_state[0],"有做空=",now_state[1],"綠atr在黃上=",now_state[2],"紅atr在黃下=",now_state[3],"有個倉位=",now_state[4])
    

花時: 3.920711040496826 
前一根黃色上軌: 20868.23686989681 
前一根黃色下軌: 20574.95008016563 
前一根綠色上ATR: 20863.67613242913 
前一根紅色下ATR: 20568.91386757087 
前一根收盤: 20716.18 
當前狀態: 做多= 0 做空= 0 綠atr在黃上= 0 紅atr在黃下= 0 有沒有倉位= 0
花時: 3.8683438301086426 
前一根黃色上軌: 20868.06235162831 
前一根黃色下軌: 20574.800713846846 
前一根綠色上ATR: 20863.67613242913 
前一根紅色下ATR: 20568.91386757087 
前一根收盤: 20716.18 
當前狀態: 做多= 0 做空= 0 綠atr在黃上= 0 紅atr在黃下= 0 有沒有倉位= 0


KeyboardInterrupt: 